# PROJECT 4 - Fatal Accident Data

In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import zipfile
import pandas as pd
import folium
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import findspark
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from mrjob.job import MRJob
import geopandas as gpd
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Specify file path
file_path = 'C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/FARS2022NationalCSV/accident_lesscolumns.csv'

In [3]:
# Read the CSV file using Pandas
df = pd.read_csv(file_path, encoding='latin1')

## CLEAN DF & CREATE NEW & CALCULATED COLUMNS FOR ANALYSIS

In [5]:
# Function to clean time columns
def clean_time(hour, minute):
    if hour >= 24 or minute >= 60:
        return '00:01'
    return f"{str(hour).zfill(2)}:{str(minute).zfill(2)}"

In [6]:
# 1. Combine columns MONTH & DAY to form a new column ACCIDENT_DATE
df['ACCIDENT_DATE'] = pd.to_datetime(df[['MONTH', 'DAY']].assign(year=2022))

# 2. Combine columns HOUR & MINUTE to form a new column TIME_OCCUR
df['TIME_OCCUR'] = df.apply(lambda row: clean_time(row['HOUR'], row['MINUTE']), axis=1)

# 3. Combine columns NOT_HOUR & NOT_MIN to form a new column TIME_REPORTED
df['TIME_REPORTED'] = df.apply(lambda row: clean_time(row['NOT_HOUR'], row['NOT_MIN']), axis=1)

# 4. Combine columns ARR_HOUR & ARR_MIN to form a new column AMBO_ARR_TIME
df['AMBO_ARR_TIME'] = df.apply(lambda row: clean_time(row['ARR_HOUR'], row['ARR_MIN']), axis=1)

# 5. Combine columns HOSP_HR & HOSP_MN to form a new column HOSP_ARR_TIME
df['HOSP_ARR_TIME'] = df.apply(lambda row: clean_time(row['HOSP_HR'], row['HOSP_MN']), axis=1)

# Helper function to calculate time difference
def calculate_time_difference(start_time, end_time):
    start = pd.to_datetime(start_time, format='%H:%M')
    end = pd.to_datetime(end_time, format='%H:%M')
    diff = end - start
    return diff

# 6. Create a calculated column REPORT_TO_AMBO_ARR_TIME
df['REPORT_TO_AMBO_ARR_TIME'] = df.apply(lambda row: calculate_time_difference(row['TIME_REPORTED'], row['AMBO_ARR_TIME']), axis=1)

# 7. Create a calculated column AMBO_TO_HOSP_ARR_TIME
df['AMBO_TO_HOSP_ARR_TIME'] = df.apply(lambda row: calculate_time_difference(row['AMBO_ARR_TIME'], row['HOSP_ARR_TIME']), axis=1)

# 8. Create a calculated column OCCUR_TO_REPORT_TIME
df['OCCUR_TO_REPORT_TIME'] = df.apply(lambda row: calculate_time_difference(row['TIME_OCCUR'], row['TIME_REPORTED']), axis=1)

# 9. Create a calculated column OCCUR_TO_HOSP_TIME
df['OCCUR_TO_HOSP_TIME'] = df.apply(lambda row: calculate_time_difference(row['TIME_OCCUR'], row['HOSP_ARR_TIME']), axis=1)

In [8]:
cleaned_df = df[['STATENAME', 'ST_CASE', 'FATALS', 'ACCIDENT_DATE', 'TIME_OCCUR', 
                 'TIME_REPORTED', 'AMBO_ARR_TIME', 'HOSP_ARR_TIME', 'REPORT_TO_AMBO_ARR_TIME', 
                 'AMBO_TO_HOSP_ARR_TIME', 'OCCUR_TO_REPORT_TIME', 'OCCUR_TO_HOSP_TIME']]

# Convert time columns to AM/PM format
time_columns = ['TIME_OCCUR', 'TIME_REPORTED', 'AMBO_ARR_TIME', 'HOSP_ARR_TIME']
for col in time_columns:
    df[col] = pd.to_datetime(df[col], format='%H:%M', errors='coerce').dt.strftime('%I:%M %p')

cleaned_df

,STATENAME,ST_CASE,FATALS,ACCIDENT_DATE,TIME_OCCUR,TIME_REPORTED,AMBO_ARR_TIME,HOSP_ARR_TIME,REPORT_TO_AMBO_ARR_TIME,AMBO_TO_HOSP_ARR_TIME,OCCUR_TO_REPORT_TIME,OCCUR_TO_HOSP_TIME
0,Alabama,10001,1,2022-01-01,12:30 PM,12:47 PM,01:04 PM,01:47 PM,0 days 00:17:00,0 days 00:43:00,0 days 00:17:00,0 days 01:17:00
1,Alabama,10002,2,2022-01-01,04:40 PM,12:01 AM,12:01 AM,12:01 AM,0 days 00:00:00,0 days 00:00:00,-1 days +07:21:00,-1 days +07:21:00
2,Alabama,10003,1,2022-01-01,01:33 AM,01:33 AM,01:50 AM,12:01 AM,0 days 00:17:00,-1 days +22:11:00,0 days 00:00:00,-1 days +22:28:00
3,Alabama,10004,1,2022-01-02,02:46 PM,02:48 PM,03:09 PM,03:44 PM,0 days 00:21:00,0 days 00:35:00,0 days 00:02:00,0 days 00:58:00
4,Alabama,10005,1,2022-01-02,06:48 PM,06:48 PM,06:54 PM,12:01 AM,0 days 00:06:00,-1 days +05:07:00,0 days 00:00:00,-1 days +05:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...
39216,Wyoming,560114,1,2022-12-27,11:27 AM,11:30 AM,11:35 AM,12:04 PM,0 days 00:05:00,0 days 00:29:00,0 days 00:03:00,0 days 00:37:00
39217,Wyoming,560115,1,2022-10-24,07:00 AM,12:01 AM,12:01 AM,12:01 AM,0 days 00:00:00,0 days 00:00:00,-1 days +17:01:00,-1 days +17:01:00
39218,Wyoming,560116,1,2022-12-21,03:00 PM,03:02 PM,03:07 PM,03:27 PM,0 days 00:05:00,0 days 00:20:00,0 days 00:02:00,0 days 00:27:00
39219,Wyoming,560117,1,2022-12-22,12:01 AM,07:24 AM,07:40 AM,12:01 AM,0 days 00:16:00,-1 days +16:21:00,0 days 07:23:00,0 days 00:00:00


In [9]:
# Filter out rows where any of the time columns have the value ":01"
cleaned_df = cleaned_df[
    (cleaned_df['TIME_OCCUR'] != '12:01 AM') &
    (cleaned_df['TIME_REPORTED'] != '12:01 AM') &
    (cleaned_df['AMBO_ARR_TIME'] != '12:01 AM') &
    (cleaned_df['HOSP_ARR_TIME'] != '12:01 AM')
]

cleaned_df

,STATENAME,ST_CASE,FATALS,ACCIDENT_DATE,TIME_OCCUR,TIME_REPORTED,AMBO_ARR_TIME,HOSP_ARR_TIME,REPORT_TO_AMBO_ARR_TIME,AMBO_TO_HOSP_ARR_TIME,OCCUR_TO_REPORT_TIME,OCCUR_TO_HOSP_TIME
0,Alabama,10001,1,2022-01-01,12:30 PM,12:47 PM,01:04 PM,01:47 PM,0 days 00:17:00,0 days 00:43:00,0 days 00:17:00,0 days 01:17:00
3,Alabama,10004,1,2022-01-02,02:46 PM,02:48 PM,03:09 PM,03:44 PM,0 days 00:21:00,0 days 00:35:00,0 days 00:02:00,0 days 00:58:00
8,Alabama,10009,1,2022-01-04,11:40 AM,11:36 AM,11:54 AM,12:41 PM,0 days 00:18:00,0 days 00:47:00,-1 days +23:56:00,0 days 01:01:00
10,Alabama,10011,1,2022-01-05,11:45 AM,11:48 AM,11:57 AM,12:30 PM,0 days 00:09:00,0 days 00:33:00,0 days 00:03:00,0 days 00:45:00
11,Alabama,10012,2,2022-01-05,08:25 PM,08:27 PM,08:43 PM,08:55 PM,0 days 00:16:00,0 days 00:12:00,0 days 00:02:00,0 days 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...
39209,Wyoming,560107,1,2022-11-30,10:25 AM,10:30 AM,10:40 AM,11:28 AM,0 days 00:10:00,0 days 00:48:00,0 days 00:05:00,0 days 01:03:00
39214,Wyoming,560112,1,2022-12-14,05:30 PM,05:33 PM,05:40 PM,06:05 PM,0 days 00:07:00,0 days 00:25:00,0 days 00:03:00,0 days 00:35:00
39216,Wyoming,560114,1,2022-12-27,11:27 AM,11:30 AM,11:35 AM,12:04 PM,0 days 00:05:00,0 days 00:29:00,0 days 00:03:00,0 days 00:37:00
39218,Wyoming,560116,1,2022-12-21,03:00 PM,03:02 PM,03:07 PM,03:27 PM,0 days 00:05:00,0 days 00:20:00,0 days 00:02:00,0 days 00:27:00


## CREATE USA MAP VISUALIZATION

In [14]:
# Create a new dataframe with only the required columns
accident_geo_df = df[['STATENAME', 'ST_CASE', 'FATALS', 'LATITUDE', 'LONGITUD']]

# Display the first 5 rows of the DataFrame
display(accident_geo_df.head())

,STATENAME,ST_CASE,FATALS,LATITUDE,LONGITUD
0,Alabama,10001,1,33.490967,-88.274083
1,Alabama,10002,2,32.087125,-86.064153
2,Alabama,10003,1,33.428331,-86.351994
3,Alabama,10004,1,32.216903,-86.400169
4,Alabama,10005,1,33.540281,-86.687389


## CREATE CA ONLY VISUALIZATION

In [17]:
# Create a new dataframe with only the required columns and filter for California
accident_geo_ca_df = df[['STATENAME', 'ST_CASE', 'FATALS', 'LATITUDE', 'LONGITUD']]
accident_geo_ca_df = accident_geo_ca_df[accident_geo_df['STATENAME'] == 'California']

# Display the first 5 rows of the DataFrame
accident_geo_ca_df

,STATENAME,ST_CASE,FATALS,LATITUDE,LONGITUD
2754,California,60001,1,38.403658,-120.641136
2755,California,60002,1,39.147700,-121.828386
2756,California,60003,1,33.816272,-116.315058
2757,California,60004,1,33.086403,-117.025567
2758,California,60005,1,38.481636,-121.408978
...,...,...,...,...,...
6858,California,64157,1,37.277867,-121.880136
6859,California,64158,2,33.555514,-117.673081
6860,California,64159,1,33.930994,-118.317778
6861,California,64160,2,33.801675,-116.410139


In [18]:
# Load US states shapefile
shapefile_path = 'C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/shapefiles/ne_110m_admin_1_states_provinces.shp'

## USA MAP VISUALIZATION

In [ ]:
# Aggregate data by state
state_data = accident_geo_df.groupby('STATENAME').agg({
    'ST_CASE': 'count',
    'FATALS': 'sum'
}).reset_index()
state_data.columns = ['STATE', 'ACCIDENTS', 'FATALITIES']

# Convert STATE column to string
state_data['STATE'] = state_data['STATE'].astype(str)

# Load US states shapefile
shapefile_path = 'C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/shapefiles/ne_110m_admin_1_states_provinces.shp'
us_states = gpd.read_file(shapefile_path)

# Convert postal column to string
us_states['postal'] = us_states['postal'].astype(str)

# Merge state data with shapefile
us_states = us_states.merge(state_data, left_on='postal', right_on='STATE', how='left')

# Initialize a map centered around the United States
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add state polygons to the map with color scale based on fatalities
folium.Choropleth(
    geo_data=us_states,
    name='choropleth',
    data=us_states,
    columns=['STATE', 'FATALITIES'],
    key_on='feature.properties.postal',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Fatalities'
).add_to(us_map)

# Add popups with number of accidents and fatalities
for _, row in us_states.iterrows():
    folium.GeoJson(
        row['geometry'],
        tooltip=folium.Tooltip(f"State: {row['name']}<br>Accidents: {row['ACCIDENTS']}<br>Fatalities: {row['FATALITIES']}")
    ).add_to(us_map)

# Save the map to an HTML file
us_map.save('C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/us_accidents_map1.html')

## CREATE USA MAP TOTALS BY STATE POPUP

In [ ]:
usa_map_df = df[['STATENAME', 'ST_CASE', 'FATALS']]
usa_map_df.head()

In [ ]:
# Count unique ST_CASE values grouped by STATENAME
accidents_count = usa_map_df.groupby('STATENAME')['ST_CASE'].nunique().reset_index()
accidents_count.columns = ['STATENAME', 'ACCIDENTS']

# Count number of fatalities grouped by STATENAME
fatalities_count = usa_map_df.groupby('STATENAME')['FATALS'].sum().reset_index()
fatalities_count.columns = ['STATENAME', 'FATALITIES']

# Merge the counts into a single dataframe
state_data = pd.merge(accidents_count, fatalities_count, on='STATENAME')

# Load US states shapefile
shapefile_path = 'C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/shapefiles/ne_110m_admin_1_states_provinces.shp'
us_states = gpd.read_file(shapefile_path)

# Merge state data with shapefile
us_states = us_states.merge(state_data, left_on='name', right_on='STATENAME', how='left')

# Initialize a map centered around the United States
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add state polygons to the map with color scale based on fatalities
folium.Choropleth(
    geo_data=us_states,
    name='choropleth',
    data=us_states,
    columns=['STATENAME', 'FATALITIES'],
    key_on='feature.properties.name',
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Fatalities'
).add_to(us_map)

# Add popups with number of accidents and fatalities
for _, row in us_states.iterrows():
    folium.GeoJson(
        row['geometry'],
        tooltip=folium.Tooltip(f"State: {row['name']}<br>Accidents: {row['ACCIDENTS']}<br>Fatalities: {row['FATALITIES']}")
    ).add_to(us_map)

# Save the map to an HTML file
us_map.save('C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/us_accidents_map2.html')

In [ ]:
# Filter out rows where any of the time columns have the value ":01"
filtered_df = cleaned_df[
    (cleaned_df['TIME_OCCUR'] != '12:01 AM') &
    (cleaned_df['TIME_REPORTED'] != '12:01 AM') &
    (cleaned_df['AMBO_ARR_TIME'] != '12:01 AM') &
    (cleaned_df['HOSP_ARR_TIME'] != '12:01 AM')
]

# Display the filtered dataframe to verify
filtered_df

In [ ]:
# Display basic information of the DataFrame
display(df.info())

In [ ]:
# # Use Selenium to take a screenshot of the map
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# driver = 'C:/Users/mmack/.wdm/drivers/chromedriver/win64/127.0.6533.99/chromedrivermanager().install()'
# driver.get(f'file:///{us_accidents_map.png}')
# #time.sleep(5)  # Give the map some time to load

# # Save the screenshot
# screenshot_path = 'C:/Users/mmack/OneDrive/Desktop/Traffic_Collisions_Project4/Mary/us_accidents_map2.png'
# driver.save_screenshot(screenshot_path)
# driver.quit()

# print(f"Map saved as image at {screenshot_path}")